In [ ]:
!pip3 install transformers

In [ ]:
# Allowed to make changes.
import collections
import json
import pandas as pd
import re
import string
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import torch
import timeit
from ast import literal_eval

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Allowed to make changes.

# Pre-processing cell. You can use this cell to pre-process input data or load
# your models.

# ground_truth = pd.read_csv('/content/drive/MyDrive/resources/sample_ground_truth.csv')
input_para = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/paragraphs.csv")
input_ques = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_input_question_1-1.csv')
theme = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sample_theme_interval (2).csv')

# print(ground_truth.shape[0])
# print()
# print(input_para)
# print()
# print(input_ques)
# print()
# print(theme)
# t...          kubernetes
# 1   2  Google originally designed Kubernetes, but the...          kubernetes
# Dummy code.
df = pd.DataFrame(columns = ['question', 'paragraph_id', 'paragraph'])

theme_dict = {}
for i in range(input_para.shape[0]):
    theme_dict[input_para.iat[i, 2].lower()] = []
for i in range(input_para.shape[0]):
    theme_dict[input_para.iat[i, 2].lower()].append({input_para.iat[i,0] : input_para.iat[i,1]})


In [ ]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
device = torch.device('cuda')
model = torch.load("/content/drive/MyDrive/Colab Notebooks/modelx5.pt", map_location=device)
question_answering_model = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)

# Allowed to make changes.
def get_theme_model(theme):

    return question_answering_model

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [ ]:
# Allowed to make changes.
def pred_theme_ans(questions, theme_model, pred_out):
  model = theme_model
  for q in questions:
    #-------------------------------------   
    # add your prediction methodology here.
    #-------------------------------------
    para_lst = theme_dict[q["theme"].lower()]
    ans = {}
    ans["question_id"] = q["id"]
    ans["paragraph_id"] = []
    ans["answers"] = []
    for p_d in  para_lst:
        answ = model(question = q["question"], context = list(p_d.values())[0])
        if answ['answer'] != "" and answ['score']>0.70:
            ans["answers"].append(answ['answer'])
            ans["paragraph_id"].append(list(p_d.keys())[0])
    
    i=1
    while i<len(ans["answers"]):
        if ans["answers"][i-1] == ans["answers"][i]:
            ans["answers"].pop(i)
        else: 
            i+=1
    if len(ans)==0:
      ans["paragraph_id"] = [-1]
      ans["answers"] = [""]
    
    pred_out.append(ans)

In [ ]:
# NOT allowed to make changes. 

# All theme prediction.
questions = json.loads(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sample_input_question_1-1.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sample_theme_interval (2).csv").to_json(orient="records"))
pred_out = []
# print(questions)
theme_inf_time = {}
for theme_interval in theme_intervals:
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000
   # in milliseconds.
#   print(execution_time*1000)
pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('/content/drive/MyDrive/sample_output_prediction.csv', index=False)

In [ ]:
# NOT allowed to make changes. 

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

In [ ]:
# NOT allowed to make changes. 

# Evaluation methodology.
metrics = {}
pred = pd.read_csv("sample_output_prediction.csv")
truth = pd.read_csv("/content/drive/MyDrive/resources/sample_ground_truth.csv")
truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv("/content/drive/MyDrive/resources/sample_input_question.csv")
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  
  if predicted_paragraph in str(truth_paragraph_id):
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
    
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
    
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1


In [ ]:
# NOT allowed to make changes. 

# Final score.
inf_time_threshold = 200.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0
# Weight would stay hidden from teams.
theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}
for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_para_score += theme_weights[theme] * inf_time_score * para_score
print (final_para_score)
print (final_qa_score)
# print(metric["true_positive"])

0.20928130088452548
0.0


In [ ]:
s = {1,2,4,4,7,8}
print(list(s)[0])

1
